## Convert a periapsis state in the database to the parametrized variables
#### Data we want 
1. $\alpha$: SC-Earth line's angle w.r.t. Sun-B1 line
2. $\beta$: velocity ratio in <b>Earth inertial</b> frame i.e., (V_circ + DV) / V_circ
    *assumption: initial DV is parallel to V_circ when leaving LEO.

In [18]:
using DifferentialEquations
using Plots
using LinearAlgebra
import ForwardDiff
import DiffResults
using AstrodynamicsBase
# import joptimise
using Printf
using JSON

In [47]:
include("../../julia-r3bp/R3BP/src/R3BP.jl")
# using R3BP
include("../src/SailorMoon.jl")   # relative path to main file of module
param3b = SailorMoon.dyanmics_parameters()

┌ Warning: Replacing docs for `Main.SailorMoon.integrate_rk4 :: Union{Tuple{ODEProblem, Real}, Tuple{ODEProblem, Real, Union{Nothing, Vector{T} where T}}, Tuple{ODEProblem, Real, Union{Nothing, Vector{T} where T}, Bool}, Tuple{ODEProblem, Real, Union{Nothing, Vector{T} where T}, Bool, Bool}}` in module `Main.SailorMoon`
└ @ Base.Docs docs\Docs.jl:240


Main.SailorMoon.dynamics_params(0.987849414390376, 0.01215058560962404, 328900.5598102475, 384748.32292972936, 375700.3437894195, 388.8212386592645, -0.9251999994040079, 0.9251999994040079, 0.07480000059599223, 0.01709689063726318, 7.601281331451572)

In [48]:
# sample data
ϕ0 = 6.073745797
ϵr = 1.00E-06
ϵv = 1.00E-06
θsf = 5.864306287
ra = 0.017263032
rp = 3.791254294
dt1 = 15.649
dt2 = 9.909
ra_state = [392.0439494254377, -2.010566944705724, 3.9994843301278626e-22, -0.06056334928312788, -0.11480722804357063, 4.38892458884967e-23, 1.0156489999987126]
rp_state = [388.81094977253866, 0.013861857271995175, 5.148354329730029e-24, -0.2916568445607961, -16.8617917342381, -3.159835154070778e-20, 1.0255579999978974]
tof = 25.558

25.558

In [49]:
θs0 = θsf - param3b.oms * tof
θm0 = π - θs0

rE = [
    param3b.as + param3b.mu2 * cos(θm0),
    param3b.mu2 * sin(θm0),
    0.0
]

# r_sc - r_E
vec = rp_state[1:3] - rE 
x_unit = [1.0, 0.0, 0.0]
α = acos(dot(vec, x_unit) / norm(vec))

if cross(x_unit, vec)[3] <= 0
    α = -α
end

# note that by definition, α_SB1 = α_EarthIne
α

2.08395170999799

In [50]:
# convert the SB1-frame state to Earth inertial 
function transform_sb1_to_EearthIne(state_earthIne::Vector, θm::Real, ωm::Real, μ2::Real, as::Real)
    cos_θm = cos(θm)
    sin_θm = sin(θm)
    
    # step 1: SunB1 -> EM rot
    state_ = state_earthIne + [-as, 0, 0, 0, 0, 0]

    rot_mat1 = inv([
        cos_θm      -sin_θm   0 0       0      0
        sin_θm      cos_θm    0 0       0      0
        0           0         1 0       0      0
        -ωm*sin_θm -ωm*cos_θm 0 cos_θm -sin_θm 0 
        ωm*cos_θm -ωm*sin_θm 0 sin_θm  cos_θm 0
        0          0         0 0       0      1
    ])
    state_emrot = rot_mat1 * state_

    # step 2: EM rot to Earth Inertial
    ωm = param3b.oml

    rot_mat2 = [
        cos_θm -sin_θm 0.0 0.0 0.0 0.0
        sin_θm cos_θm 0.0 0.0 0.0 0.0
        0.0 0.0 1.0 0.0 0.0 0.0
        -ωm*sin_θm -ωm*cos_θm 0.0 cos_θm -sin_θm 0.0
        ωm*cos_θm -ωm*sin_θm 0.0 sin_θm cos_θm 0.0
        0.0 0.0 0.0 0.0 0.0 1.0
    ]
    state_ = rot_mat2 * state_emrot
    
    state_EarthIne = state_ + [μ2*cos(θm), μ2*sin(θm), 0, 0, 0, 0]

    
    return state_EarthIne
end


transform_sb1_to_EearthIne (generic function with 1 method)

In [60]:
# test function 
state_Eine = transform_sb1_to_EearthIne(rp_state[1:6], θm0, param3b.oml, param3b.mu2, param3b.as)


state_ = SailorMoon.transform_earthIne_to_EMrot(state_Eine, θm0, param3b.oml, param3b.mu2)
state_ = SailorMoon.transform_EMrot_to_SunB1(state_, π-θm0, param3b.oms, param3b.as)

state_2 = SailorMoon.transform_EearthIne_to_sb1(state_Eine, θm0, param3b.oml, param3b.mu2, param3b.as)

println("rp_state: ", rp_state[1:6])
println("state_: ", state_)
println("state_2: ", state_2)


state_Eine: [-0.0062991714035335366, 0.002384971705618282, 5.148354329730029e-24, -0.2916568445607952, -16.861791734238103, -3.159835154070778e-20]
state_em: [-0.01647168993313827, -0.005166788928628796, 5.148354329730029e-24, 15.82632713935098, -5.796913266618815, -3.159835154070778e-20]
rp_state: [388.81094977253866, 0.013861857271995175, 5.148354329730029e-24, -0.2916568445607961, -16.8617917342381, -3.159835154070778e-20]
state_: [388.81094977253866, 0.013861857271995175, 5.148354329730029e-24, -0.2916568445607952, -16.861791734238107, -3.159835154070778e-20]
state_2: [388.81094977253866, 0.013861857271995175, 5.148354329730029e-24, -0.2916568445607952, -16.861791734238107, -3.159835154070778e-20]


In [53]:
# take the velocity ratio β
r_parking = (200+6375)/param3b.lstar
v_LEO = sqrt(param3b.mu1 / r_parking)
v_SC_dep = norm(state_Eine[4:6]) 

β = v_SC_dep / v_LEO

2.218108649735122

In [58]:
function param_to_Earth_ini(r::Real, α::Real, β::Real, θm::Real, μ::Real)
    # build an initial state in Earth inertial frame
    state_i_EIne = [
        r*cos(α), 
        r*sin(α), 
        0,
        -β*sqrt(μ / r) * sin(α),
        β*sqrt(μ / r) * cos(α),
        0
    ] 
    println("state_EIne: ", state_i_EIne)
    
    # EarthIne -> Sunb1
    state_i_sb1 = SailorMoon.transform_EearthIne_to_sb1(state_i_EIne, θm, param3b.oml, param3b.mu2, param3b.as)
    
    return state_i_sb1
end

param_to_Earth_ini (generic function with 1 method)

In [62]:
# regenerate the trajectory arc from here
# first, in reverse way we need to recreate the initial state (at SB1 frame)
println("state_Eine: ", state_Eine)

state_i = param_to_Earth_ini(r_parking, α, β, θm0, param3b.mu1)

state_Eine: [-0.0062991714035335366, 0.002384971705618282, 5.148354329730029e-24, -0.2916568445607952, -16.861791734238103, -3.159835154070778e-20]
state_EIne: [-0.008389525611994889, 0.014888014337931325, 0.0, -14.692186569010545, -8.279174960416393, 0.0]
state_em: [-0.028967870168013762, -0.0030357949540299346, 0.0, 2.9930552598999602, -16.56927928404329, 0.0]


6-element Vector{Float64}:
 388.8088594183302
   0.026364899904308223
   0.0
 -14.692186569010547
  -8.279174960416395
   0.0

In [ ]:
14.7^2+8.27